### Задача идентификации взломщика по его поведению в сети Интернет

Ссылка: [Catch Me If You Can](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2)

У нас есть данные по посещениям пользователями каких-то сайтов и времени посещения.

Необходимо определить сессии в тесте, которые осуществил определенный юзер. Его класс в трейне 1, все остальные юзеры 0.

В ноутбуке приведен алгоритм создания спарс матрицы из сайтов, которые посещали люди из выборки. В каждой строке будет от 1 до 10 непустых элементов.

По времени никаких фич не построено, это для самостоятельной работы.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score as auc
from sklearn.linear_model import LogisticRegression
from scipy.sparse import lil_matrix
from sklearn.model_selection import KFold
import time


%matplotlib inline
pd.set_option('display.max_columns', None)

In [ ]:
data = pd.read_csv('../data/Alice/train_sessions.csv')
test = pd.read_csv('../data/Alice/test_sessions.csv')

In [ ]:
#запоминаем индекс для последующего разделения трейна и теста
idx_split = data.shape[0]
#объединяем трейн и тест. Создание спарс матрицы в таком, как у нас, виде особо не ликует.
data = data.append(test, sort=False).reset_index(drop=True)

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
num_cols = [col for col in data.columns if data[col].dtype=='int64' or
                                           data[col].dtype=='float64']
num_cols.remove('target')
time_cols = [col for col in data.columns if data[col].dtype=='object']

In [ ]:
for col in time_cols:
    data[col] = pd.to_datetime(data[col], yearfirst=True)

In [ ]:
#Заполним отсутствующие сайты уникальным значением.
data[num_cols] = data[num_cols].fillna(-1)
#Это необходимо для того, чтобы данные по сайтам привести к целочисленному типу.
data[num_cols] = data[num_cols].astype(int)

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
sites = [col for col in data.columns if col.startswith('site')]

In [ ]:
# Этой функцией создаем словарь посещенных юзерами сайтов
def find_sites(li):
    lli = {}
    for l in li:
        if l > 0:
            if l in lli:
                lli[str(l)] += 1
            else:
                lli[str(l)] = 1
    return lli

In [ ]:
data['all_sites'] = data[sites].apply(find_sites, axis=1)

In [ ]:
data.head()

In [ ]:
#Считаем, сколько всего сайтов было пройдено за сессию
data['len_sites'] = data['all_sites'].apply(lambda x: sum(x.values()))

In [ ]:
#для скорости создаем лист из наших словарей
sp_list = list(data['all_sites'])

In [ ]:
#инициализируем пустую спарс матрицу и задаем ее размер с запасом
site1 = lil_matrix((data.shape[0], 100000))#, dtype=np.int8)
row = 0

#в цикле идем по каждой строке и ставим единичку в ту колонку, сайт которой есть в строке
for s in sp_list:
    for key, value in s.items():
        site1[row, key] = 1
    row+=1

#убираем лишние нулевые колонки
site1 = site1.tocsc()[:, np.where(site1.getnnz(axis=0) > 0)[0]].tocsr()


In [ ]:
#размер нашего спарса
site1

In [ ]:
#в данной ячейке мы удаляем все колонки с нулевыми значениями в колонках теста 
#и здесь мы ликуем, надеясь, что это даст выше скор
ttest = site1[idx_split:]
site1 = site1.tocsc()[:, np.where((ttest.getnnz(axis=0) > 0))[0]].tocsr()

In [ ]:
#делим снова на тест и трейн
ttest = site1[idx_split:]
site1 = site1[:idx_split]

In [ ]:
site1

In [ ]:
ttest

In [ ]:
#обучаем с кросс-валидацией линейную регрессию
#предсказываем 10 раз трейн, дальше усредним
answ = []
v_metric = []

n=1

kf = KFold(n_splits=10, shuffle=True, random_state=777)   
for tr_ind, val_ind in kf.split(site1):
    print('Start {} fold'.format(n))

    val = site1[val_ind]
    ttt = site1[tr_ind] 

    start_time = time.time()
    clf = LogisticRegression(C=2, solver='lbfgs', max_iter=1000,
                            random_state=777)

    clf.fit(ttt, data['target'][tr_ind].reset_index(drop=True)) 

    model_pred_valid = clf.predict_proba(val)[:, 1]

    y_valid = data['target'][val_ind].reset_index(drop=True)
    valid_metric = auc(y_valid, model_pred_valid)
    v_metric.append(valid_metric)

    print('fold score:', valid_metric, round((time.time() - start_time)/60, 2))
    model_pred = clf.predict_proba(ttest)[:, 1]
    answ.append(model_pred)

    n+=1

    print('crossval score:', np.mean(v_metric), 'std', np.std(v_metric))
    print('---------------------------------------')

In [ ]:
#собираем предсказания теста
answ_df = pd.DataFrame()
for i in range(len(answ)):
    answ_df['an'+str(i)] = answ[i]
answ_df['answer'] = answ_df.mean(axis=1)

In [ ]:
answ_df.head()

In [ ]:
#пишем функцию для сабмита и делаем сабмит

def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
y_test = np.array(answ_df['answer'])

In [ ]:
write_to_submission_file(y_test, 'submission.csv')